In [1]:

import pandas as pd

from utils.io import yaml_to_dict,dict_to_yaml

from cleanco import basename

import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict

# from projects.lobbying.postproc import terms_present
import matplotlib.colors as mc

import warnings

warnings.filterwarnings("ignore")
%load_ext jupyter_black

In [5]:
def invert_sector_dict(sectors_path) -> Dict[str, str]:
    """reads in the sector assignment yaml to dict; inverts dict s.t. each company is a key"""
    sector_assignments = yaml_to_dict(sectors_path)

    all_companies = []
    for _, value in sector_assignments.items():
        all_companies = all_companies + value
    # print(all_companies)

    company_sector_dict = {}
    for k, vv in sector_assignments.items():
        for v in vv:
            company_sector_dict = company_sector_dict | {v: k}

    return company_sector_dict

In [6]:
df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct4_compiled_filings.csv")
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)

In [ ]:
for i in df.client_rename.unique():
    if i not in inverted_sector_dict.keys():
        print(f"{i}")

In [3]:
df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct4_compiled_filings.csv")

In [207]:
df = pd.read_csv(
    "/Volumes/Samsung_T5/data/lobbying/oct5_postprocessed_filings.csv"
)

In [7]:
portland_replace_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/company_name_replacements.yml"
)

df["client_rename"] = [
    portland_replace_dict[x] if x in list(portland_replace_dict.keys()) else r
    for x, r in zip(df.client_name, df.client_rename)
]
df["sector"] = [inverted_sector_dict[x] for x in df["client_rename"]]
sectors_dict = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)
inverted_sector_dict = invert_sector_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)
companies = [c.rstrip().lstrip() for c in list(inverted_sector_dict.keys())]
not_assigned = sorted(list(set(df.client_rename.unique()) - set(companies)))
for c in not_assigned:
    # for c in ["FMC"]:
    names = list(df.loc[df.client_rename == c].client_name.unique())
    for n in names:
        print(f"{n}: {c}")
    # descriptions = list(df.loc[df.client_rename == c].clean_description.unique())
    # for d in descriptions:
    #    print(d)
    print("")

In [8]:
df.clean_description = df.clean_description.fillna("")
df["ccs"] = [
    1
    if (
        (" ccs" in x.lower())
        | (" ccus" in x.lower())
        | ("capture and utilization" in x.lower())
        | ("carbon capture storage" in x.lower())
        | ("carbon capture and storage" in x.lower())
        | ("co2 capture and sequestration" in x.lower())
        | ("co2 capture sequestration" in x.lower())
        | ("capture transport and storage" in x.lower())
        | ("capture and storage of carbon" in x.lower())
        | ("capture and sequestration of carbon" in x.lower())
        | ("capture sequestration of carbon" in x.lower())
        | ("carbon capture and sequestration" in x.lower())
        | ("carbon capture sequestration" in x.lower())
        | ("co2 capture storage" in x.lower())
        | ("clean hydrogen" in x.lower())
        | ("blue hydrogen" in x.lower())
        | ("beccs" in x.lower())
        | ("hydrogen hub" in x.lower())
        | ("carbon dioxide pipeline" in x.lower())
        | ("co2 pipeline" in x.lower())
        | ("class vi" in x.lower())
    )
    else 0
    for x in df.clean_description
]
df["bio"] = [
    1
    if (
        ("biofuel" in x.lower())
        | ("biomass" in x.lower())
        | ("bioenergy" in x.lower())
        | ("waste to energy" in x.lower())
        | ("wastetoenergy" in x.lower())
        | ("sustainable aviation" in x.lower())
        | ("biodiesel" in x.lower())
    )
    else 0
    for x in df.clean_description
]
df.ccs.value_counts()

ccs
0    217087
1      5057
Name: count, dtype: int64

In [9]:
df.loc[df.ccs == 1].sector.value_counts()

sector
investor owned utilities                   894
oil and gas                                841
ccs                                        427
coal                                       333
municipal and cooperative utilities        276
labor union                                239
biofuels                                   226
chemicals, refining, and plastics          226
pipelines                                  192
energy technology                          105
environmental                               92
clean hydrogen                              91
railway                                     91
business advocacy                           89
independent power producers                 89
mining and metals                           70
cement                                      67
paper products                              65
fuel cell                                   62
finance                                     61
green hydrogen                              57
renewa

In [14]:
for c in sorted(
    list(
        set(
            (df.loc[(df.sector == "remove these organizations")].client_rename.unique())
        )
    )
):
    print(f"- {c}")

- 4SSILVERSWORD SOFTWARE AND
- 7ELEVEN
- A O SMITH
- ABIR
- ACA INTERNATIONAL
- ACACONNECTS
- ACADEMY OF MODEL AERONAUTICS
- ACCREDITING COMMISSION FOR COLLEGES AND JUNIOR COLLEGES WASC
- ACT FOR NIH
- ACXIOM
- ADAPDIX
- ADVANCED DRAINAGE SYSTEMS
- ADVANCED SYSTEMS TECHNOLOGY
- ADVOCACY COUNCIL OF ACAAI
- AERONAUTICAL REPAIR STATION ASSOCIATION
- AFFINITY DIRECT
- AICPA ASSOCIATION OF INTERNATIONAL CERTIFIED PROFESSIONAL ACCOUNTANTS
- AIM
- AIRBNB
- AIREON
- AKER BIOMARINE
- ALICE TECHNOLOGIES
- ALLIANCE FOR AGING RESEARCH
- ALLIANCE FOR AMERICAN ADVERTISING
- ALLIANCE FOR BANGLADESH WORKER SAFETY
- ALLIANCE FOR DRONE INNOVATION
- ALLIANCE FOR RETIRED AMERICANS
- ALLIANCE FOR SAFETY AND JUSTICE
- ALS ASSOCIATION
- ALTAFIBER AND ITS SUBSIDIARIES
- ALTERNATIVE DIRECT INVESTMENT SECURITIES ASSOCIATION
- AMAC ACTION
- AMEDISYS
- AMERICA OUTDOORS ASSOCIATION
- AMERICAN ADVERTISING FEDERATION
- AMERICAN ASSOCIATION FOR STUDY OF LIVER DISEASES
- AMERICAN ASSOCIATION FOR TREATMENT OF OPIOID DE

In [13]:
for c in sorted(
    list(
        df.loc[
            (df.ccs == 1)
            & (df.sector == "remove these organizations")
            & (df.sector != "ccs")
        ].client_rename.unique()
    )
):
    print(f"- {c}")

- BCE


In [ ]:
df.loc[
    (df.client_rename == "AMERICANS FOR TAX REFORM") & (df.ccs == 1)
].clean_description.values

In [ ]:
for i in list(
    df.loc[df.client_rename == "RENEWABLE ENERGY GROUP"].client_name.unique()
):
    print(f"{i}: ")

In [93]:
import os


def rename_files(directory):
    # Iterate over all files in the specified directory
    for filename in os.listdir(directory):
        # Check if the file is a CSV file
        if filename.endswith(".csv"):
            # Construct the old file path
            old_file_path = os.path.join(directory, filename)

            # Construct the new file name with 'latest_' prefix
            new_file_name = "latest_" + filename

            # Construct the new file path
            new_file_path = os.path.join(directory, new_file_name)

            # Rename the file
            os.rename(old_file_path, new_file_path)
            print(f"Renamed: {filename} to {new_file_name}")



In [ ]:
tmp = yaml_to_dict(
    "/Users/lindseygulden/dev/leg-up-private/projects/lobbying/sectors_updated.yml"
)
he = tmp["chemicals, refining, and plastics"]
he = sorted(list(set(he)))
for h in he:
    print(f"- {h}")

In [ ]:
for c in not_assigned:
    original_names = df.loc[df.client_rename == c].client_name.unique()
    for n in original_names:
        print(f"{n}: {c}")

In [ ]:
# df.to_csv("~/Desktop/postprocessed_ccs.csv")

df = pd.read_csv("/Volumes/Samsung_T5/data/lobbying/oct4_postprocessed_filings.csv")
df[
    [
        "ccs_company",
        "contains_ccs_description",
        "ccs_bills",
        "ccs_bills_number_only",
        "bill_with_ccs_term",
        "very_likely_ccs",
        "likely_ccs",
        "potentially_ccs",
    ]
].sum()

In [ ]:
maybe_df = df.loc[
    (df.contains_ccs_description == 1)
    & (df.ccs_bills == 0)
    & (df.ccs_bills_number_only == 0)
    & (df.ccs_company == 0)
    & (df.clean_h2_description == 0)
    & (df.clean_h2_company == 0)
]


maybe_df["obvi"] = [
    1
    if (
        (
            ("hydrogen" in x.lower())
            and (
                s
                in [
                    "clean hydrogen",
                    "oil and gas",
                    "pipelines",
                    "oilfield services",
                    "chemicals, refining, and plastics",
                ]
            )
        )
        | ("ccs" in x.lower())
        | ("capture co2" in x.lower())
        | ("carbon capute" in x.lower())
        | ("carbon capture" in x.lower())
        | ("carboncapture" in x.lower())
        | ("capture carbon dioxide" in x.lower())
        | ("carbon dioxide capture" in x.lower())
        | ("capture and sequestration of carbon dioxide" in x.lower())
        | ("capture and sequestration of co2" in x.lower())
        | ("capture and storage of carbon dioxide" in x.lower())
        | ("capture sequestration of carbon dioxide" in x.lower())
        | ("capture sequestration of co2" in x.lower())
        | ("capture storage of carbon dioxide" in x.lower())
        | ("renewable natural gas hydrogen" in x.lower())
        | ("co2 capture" in x.lower())
        | ("clean coal sequestration" in x.lower())
        | ("blue hydrogen" in x.lower())
        | ("clean hydrogen" in x.lower())
        | ("ccs" in x.lower())
        | ("ccus" in x.lower())
        | ("primacy" in x.lower())
        | ("capture and storage" in x.lower())
        | ("capture of carbon dioxide" in x.lower())
        | ("carbon sequestration" in x.lower())
        | ("carbon dioxide utilization" in x.lower())
        | ("carbon dioxide sequestration" in x.lower())
        | ("co2 sequestration" in x.lower())
        | ("co2 storage" in x.lower())
        | ("petra nova" in x.lower())
        | ("futuregen" in x.lower())
        | (("section 45" in x.lower()) and ("coal" in x.lower()))
        | ("enhanced oil recovery" in x.lower())
        | ("greenhouse gas capture and storage" in x.lower())
        | ("greenhouse gas capture storage" in x.lower())
        | (" eor " in x.lower())
        | ("carbon dioxide storage" in x.lower())
        | ("45q" in x.lower())
        | ("carbon storage" in x.lower())
        | ("45 q" in x.lower())
        | ("45v" in x.lower())
        | ("45 v" in x.lower())
        | ("45z" in x.lower())
        | ("45 z" in x.lower())
        | ("carbon utilization" in x.lower())
        | ("co2 geologic sequestration" in x.lower())
        | ("sequester co2" in x.lower())
        | ("injection of carbon dioxide" in x.lower())
        | ("captures and sequesters carbon dioxide" in x.lower())
        | ("sequester carbon dioxide" in x.lower())
        | ("storage of co2" in x.lower())
        | ("hydrogen hub" in x.lower())
        | ("carbon dioxide pipeline" in x.lower())
        | ("pipeline safety" in x.lower())
        | ("co2 pipeline" in x.lower())
        | ("carbon dioxide emmissions sequestration" in x.lower())
        | ("class vi" in x.lower())
        | ("renewable natural gas and hydrogen" in x.lower())
        | ("hydrogen tax credit" in x.lower())
        | ("carbon oxide capture" in x.lower())
        | ("carbon oxide sequestration" in x.lower())
        | ("conversion use and storage of carbon dioxide" in x.lower())
        | ("h2 hub" in x.lower())
    )
    else 0
    for x, s in zip(maybe_df.clean_description, maybe_df.sector)
]
print(maybe_df.obvi.value_counts())
for i, row in maybe_df.iterrows():
    if row["obvi"] == 0:
        print(f"{row['client_rename']}: {row['clean_description']}")
        for t in [
            "natural gas",
            "hydrogen",
            "air capture",
            "storage",
            "co2",
            "carbon dioxide",
            "injection",
            "pipeline",
            "co2 removal",
            "emissions",
            "pipe line",
            "oil recovery",
            "clean coal",
            "methane",
            "storage",
            "low carbon",
            "zero",
            "capture",
            "EPA",
            "UIC",
            "injection",
            "underground injection control",
            "inflation reduction act",
            "IIJA",
            "sequestration",
        ]:
            if t.lower() in row["clean_description"].lower():
                print(f"    {t}")

In [ ]:
maybe_df = df.loc[
    (df.ccs_because_of_who_says_it == 1)
    & (df.very_likely_ccs == 0)
    & (df.bill_with_ccs_term == 0)
]

maybe_df["obvi"] = [
    1
    if (
        ("hydrogen" in x.lower())
        | ("sustainable aviation" in x.lower())
        | ("carbon management" in x.lower())
        | ("section 45" in x.lower())
    )
    else 0
    for x, s in zip(maybe_df.clean_description, maybe_df.sector)
]
print(maybe_df.obvi.value_counts())
print("")
for i, row in maybe_df.iterrows():
    if row["obvi"] == 0:
        print(f"{row['client_rename']}: {row['clean_description']}")
        for t in [
            "natural gas",
            "hydrogen",
            "air capture",
            "storage",
            "co2",
            "carbon dioxide",
            "injection",
            "pipeline",
            "co2 removal",
            "emissions",
            "pipe line",
            "oil recovery",
            "clean coal",
            "methane",
            "storage",
            "low carbon",
            "zero",
            "capture",
            "EPA",
            "UIC",
            "injection",
            "underground injection control",
            "inflation reduction act",
            "IIJA",
            "sequestration",
        ]:
            if t.lower() in row["clean_description"].lower():
                print(f"    {t}")

In [359]:
df = df.loc[df.very_likely_ccs == 1]

tot_dollars = df.lobbying_activity_usd.sum()

tot_contacts = df.n_entities_lobbied.sum()

df["pct_of_lobbying_dollars"] = [
    100 * x / tot_dollars for x in df.lobbying_activity_usd
]

df["pct_of_entity_contacts"] = [100 * x / tot_contacts for x in df.n_entities_lobbied]

In [ ]:
lobbying_by_sector_totals_df = (
    df[
        [
            "sector",
            "lumped_sector",
            "very_lumped_sector",
            "lobbying_activity_usd",
            "n_entities_lobbied",
            "pct_of_lobbying_dollars",
            "pct_of_entity_contacts",
        ]
    ]
    .loc[df["very_likely_ccs"] == 1]
    .groupby(["very_lumped_sector"])
    .agg(
        {
            # "lumped_sector": "first",
            "very_lumped_sector": "first",
            "lobbying_activity_usd": sum,
            "n_entities_lobbied": sum,
            "pct_of_lobbying_dollars": sum,
            "pct_of_entity_contacts": sum,
        }
    )
    .sort_values(by="lobbying_activity_usd", ascending=False)
)


lobbying_by_sector_totals_df  # .to_clipboard()

In [ ]:
lobbying_by_sector_totals_df.sum()

In [157]:
def terms_present(phrase, term_list, find_any=True):
    """utility function to see if terms in terms_list are present in a given phrase
    Args:
        phrase: phrase to be searched
        term_list: list of strings to be searched for within phrase
        find_any: boolean -- true if function should return true if any of the terms are in the phrase;
            false if the function should only return true if all terms are present
    Returns:
        int: 1 if one or more of the terms are present in phrase, 0 otherwise
    """
    if not isinstance(phrase, str):
        raise TypeError("phrase must be a string")
    if not isinstance(term_list, list):
        raise TypeError(
            "term_list must be a list of strings (even if it is a list of length 1)"
        )
    if phrase is None:
        return 0
    n_present = 0

    for term in term_list:
        if isinstance(term, list):
            if terms_present(phrase, term, find_any=False):
                if find_any:
                    return 1
                n_present += 1
        elif term.lower() in phrase.lower():
            if find_any:
                return 1
            n_present += 1
    if n_present == len(term_list):
        return 1
    return 0


def substitute(
    x: str,
    use_basename: bool = False,
    re_types: str = r"[^\w\s]",
    replace_str: str = "",
):
    """wrapper function for regular expression substitute funciton, linked with basename lib"""
    # use basename for company names
    if not isinstance(x, str):
        x = ""
    if use_basename:
        return basename(re.sub(re_types, replace_str, x))
    # don't use basename for general strings
    return re.sub(re_types, replace_str, x).rstrip().lstrip()

In [366]:
def quarter_to_decimal(q: str) -> float:
    """converts string quarter to decimal fraction of year"""
    if q == "second_quarter":
        return 135 / 365.25
    if q == "third_quarter":
        return 227 / 365.25
    if q == "first_quarter":
        return 46 / 365.25
    if q == "fourth_quarter":
        return 319 / 365.25
    if q == "year_end":
        return 274 / 265.25
    if q == "mid_year":
        return 91 / 365.25
    raise ValueError("%s quarter string is not one of accepted strings.", q)

In [367]:
df["decimal_filing_year"] = [
    float(y) + quarter_to_decimal(q) for y, q in zip(df.filing_year, df.filing_period)
]

In [368]:
# merge client states
df.loc[df.client_state != df.client_ppb_state].client_state.value_counts(dropna=False)
df["client_state_merged"] = df.client_state.fillna(df["client_ppb_state"])

In [ ]:
names = df.loc[df.sector == "environmental"].client_rename.unique()
for name in names:
    original_names = list(df.loc[df.client_rename == name].client_name.unique())
    for o in original_names:
        print(f"{o}: {name}")

In [374]:
univ_df = pd.read_csv("~/Desktop/data/university_with_rename.csv", index_col=[0])
univ_df.institution_name = [x.upper() for x in univ_df.institution_name]
univ_df.to_csv("university_with_rename.csv")

In [ ]:
ccs_univ_df = (
    df[["client_rename", "lobbying_activity_usd"]]
    .loc[df.sector == "higher education"]
    .groupby(["client_rename"])
    .sum()
    .sort_values(by=["lobbying_activity_usd"], ascending=False)
)
univ_df.merge(
    ccs_univ_df, right_index=True, left_on="institution_name", how="outer"
).sort_values(by="total_og_funding_2010_to_2020", ascending=False)

In [ ]:
df.loc[df.client_rename == "SCANA"].client_name.unique()

In [ ]:
df.columns.values

In [ ]:
test_df = (
    df[["lobbying_activity_usd", "dollars_spent_lobbying", "filing_uuid"]]
    .groupby("filing_uuid")
    .agg({"dollars_spent_lobbying": "first", "lobbying_activity_usd": "sum"})
)
test_df.columns = ["filing_usd", "ccs_lobbying_usd"]
test_df.loc[test_df.filing_usd < test_df.ccs_lobbying_usd]

In [ ]:
for i in sorted(list(df.loc[df.sector == "biofuels"].client_rename.unique())):
    print(i)

In [ ]:
df[["filing_year", "client_rename", "lobbying_activity_usd"]].loc[
    df.sector == "land transit"
].groupby(["client_rename"]).sum().sort_values(
    # ].groupby(["filing_year", "client_rename"]).sum().sort_values(
    by=["lobbying_activity_usd"],
    ascending=False,
).head(
    60
)

In [ ]:
# for histogram hue ordering:
sector_for_group = "very_lumped_sector"
very_lumped_order = list(
    df[[sector_for_group, "lobbying_activity_usd"]]
    .groupby(sector_for_group)
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
    .index
)

# lobbying_activity_usd
years = list(range(2004, 2025))

tmp_df = (
    df[["filing_year", "lobbying_activity_usd", sector_for_group]]
    .loc[(df.very_likely_ccs == 1)]
    .groupby([sector_for_group, "filing_year"])
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
    .reset_index()
).copy(deep=True)
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
ax = sns.histplot(
    data=tmp_df,
    x="filing_year",
    bins=[0.5 + x for x in years],
    hue=sector_for_group,
    hue_order=very_lumped_order,
    shrink=0.9,
    weights="lobbying_activity_usd",
    multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
# ax.legend(ncols=3, loc="upper center")a

In [ ]:
tmp_df = (
    df[
        [
            "filing_year",
            "filing_period",
            "decimal_filing_year",
            "n_entities_lobbied",
            "lobbying_activity_usd",
        ]
    ]
    .loc[df.client_rename == "EXXON"]
    .groupby(["filing_year", "filing_period"])
    .agg(
        {
            "decimal_filing_year": "mean",
            "n_entities_lobbied": sum,
            "lobbying_activity_usd": sum,
        }
    )
    .reset_index()
    .sort_values(by="decimal_filing_year", ascending=True)
)
sns.pointplot(data=tmp_df, x="decimal_filing_year", y="lobbying_activity_usd")

In [ ]:
# lobbying_activity_usd
years = list(range(2004, 2025))
for agency in [
    "environmental protection agency (epa)",
    "energy, dept of",
    "treasury, dept of",
    "house of representatives",
    "senate",
]:
    tmp_df = (
        df[["filing_year", "lobbying_activity_usd", "very_lumped_sector"]]
        .loc[(df.very_likely_ccs == 1) & (df[agency] == 1)]
        .groupby(["very_lumped_sector", "filing_year"])
        .sum()
        .sort_values(by="lobbying_activity_usd", ascending=False)
        .reset_index()
    ).copy(deep=True)
    fig, ax = plt.subplots(1, 1, figsize=(17, 5))
    ax = sns.histplot(
        data=tmp_df,
        x="filing_year",
        bins=[0.5 + x for x in years],
        hue="very_lumped_sector",
        hue_order=very_lumped_order,
        shrink=0.9,
        weights="lobbying_activity_usd",
        multiple="stack",
    )
    ax.set_xticks(years)
    ax.set_xticklabels([str(x) for x in years], fontsize=12)
    ax.set_axisbelow(True)
    ax.yaxis.grid()
    ax.set_ylabel("100 million USD", fontsize=14)
    ax.set_xlabel("")
    ax.set_title(agency)
    # ax.legend(ncols=3, loc="upper center")a

In [ ]:
# lobbying_activity_usd
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
years = list(range(2004, 2025))

tmp_df = (
    df[
        [
            "filing_year",
            "legistlative_entities_lobbied",
            "executive_entities_lobbied",
            "n_entities_lobbied",
        ]
    ]
    .loc[(df.very_likely_ccs == 1)]
    .groupby(["filing_year"])
    .sum()
    .reset_index()
).copy(deep=True)

tmp_df = tmp_df.melt(id_vars="filing_year")
ax = sns.lineplot(
    data=tmp_df,
    x="filing_year",
    # bins=[0.5 + x for x in years],
    # hue="very_lumped_sector",
    # shrink=0.9,
    y="value",
    hue="variable"
    # multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
ax.legend()
# ax.legend(ncols=3, loc="upper center")a

In [213]:
entities = [
    "access board",
    "administration for children & families (acf)",
    "administration on aging",
    "advisory council on historic preservation (achp)",
    "african development foundation",
    "agency for healthcare research & quality (ahrq)",
    "agency for toxic substances & disease registry",
    "agriculture, dept of (usda)",
    "air force, dept of",
    "alcohol & tobacco tax & trade bureau (ttb)",
    "alternative agricultural research & commercialization center",
    "american battle monuments commission",
    "amtrak - natl railroad passenger corporation",
    "appalachian regional commission",
    "architect of the capitol (aoc)",
    "architectural & transportation barriers compliance board",
    "army, dept of (corps of engineers)",
    "army, dept of (other)",
    "ballistic missile defense organization",
    "bureau of alcohol tobacco firearms & explosives",
    "bureau of arms control",
    "bureau of citizenship & immigration services (bcis)",
    "bureau of economic analysis",
    "bureau of engraving & printing",
    "bureau of indian affairs (bia)",
    "bureau of industry and security (bis)",
    "bureau of justice assistance",
    "bureau of justice statistics",
    "bureau of labor statistics (bls)",
    "bureau of land management (blm)",
    "bureau of prisons (bop)",
    "bureau of reclamation",
    "bureau of the census",
    "bureau of the public debt",
    "bureau of transportation statistics",
    "centers for disease control & prevention (cdc)",
    "centers for medicare and medicaid services (cms)",
    "central intelligence agency (cia)",
    "commerce, dept of (doc)",
    "commodity futures trading commission (cftc)",
    "congressional budget office (cbo)",
    "consumer financial protection bureau (cfpb)",
    "consumer product safety commission (cpsc)",
    "corporation for natl & community service",
    "corporation for public broadcasting",
    "corrections program office",
    "council of economic advisers (cea)",
    "council on environmental quality (ceq)",
    "defense advanced research projects agency (darpa)",
    "defense business transformation agency (bta)",
    "defense commissary agency",
    "defense contract audit agency (dcaa)",
    "defense department field activities",
    "defense finance & accounting service (dfas)",
    "defense information systems agency",
    "defense intelligence agency (dia)",
    "defense investigative service (dis)",
    "defense legal services agency",
    "defense logistics agency",
    "defense nuclear facilities safety board",
    "defense security assistance agency",
    "defense security cooperation agency (dsca)",
    "defense special weapons agency",
    "defense threat reduction agency (dtra)",
    "defense, dept of (dod)",
    "director of national intelligence (odni)",
    "drug court program office",
    "drug enforcement administration (dea)",
    "economic development administration",
    "education, dept of",
    "election assistance commission (eac)",
    "employment & training administration",
    "employment standards administration",
    "energy, dept of",
    "environmental protection agency (epa)",
    "equal employment opportunity commission (eeoc)",
    "executive office of the president (eop)",
    "export-import bank of the united states (exim bank)",
    "farm credit administration",
    "federal accounting standards advisory board (fasab)",
    "federal aviation administration (faa)",
    "federal bureau of investigation (fbi)",
    "federal communications commission (fcc)",
    "federal deposit insurance corporation (fdic)",
    "federal election commission (fec)",
    "federal emergency management agency (fema)",
    "federal energy regulatory commission (ferc)",
    "federal highway administration (fha)",
    "federal housing finance agency (fhfa)",
    "federal labor relations authority (flra)",
    "federal law enforcement training center",
    "federal management service",
    "federal maritime commission",
    "federal mediation & conciliation service",
    "federal mine safety health review commission (fmshrc)",
    "federal motor carrier safety administration",
    "federal railroad administration",
    "federal reserve system",
    "federal retirement thrift investment board",
    "federal trade commission (ftc)",
    "federal transit administration (fta)",
    "financial crimes enforcement network (fincen)",
    "financial management service (fms)",
    "food & drug administration (fda)",
    "general services administration (gsa)",
    "ginnie mae",
    "government accountability office (gao)",
    "government printing office (gpo)",
    "health & human services, dept of (hhs)",
    "health resources & services administration (hrsa)",
    "homeland security, dept of (dhs)",
    "house of representatives",
    "housing & urban development, dept of (hud)",
    "indian health service",
    "institute of museum and library services (imls)",
    "inter-american foundation",
    "interior, dept of (doi)",
    "internal revenue service (irs)",
    "intl trade administration (ita)",
    "job corps",
    "joint chiefs of staff",
    "joint service schools",
    "justice, dept of (doj)",
    "labor, dept of (dol)",
    "legal services corporation",
    "library of congress (loc)",
    "marine mammal commission",
    "maritime administration (marad)",
    "medicare payment advisory commission (medpac)",
    "merit systems protection board",
    "mine safety & health administration (msha)",
    "mine safety & health review commission",
    "minerals management service",
    "minority business development agency",
    "natl aeronautics & space administration (nasa)",
    "natl archives & records administration (nara)",
    "natl bioethics advisory commission",
    "natl capital planning commission (ncpc)",
    "natl commission libraries & information science (ncls)",
    "natl council on disability",
    "natl credit union administration (ncua)",
    "natl economic council (nec)",
    "natl endowment for the arts",
    "natl endowment for the humanities",
    "natl foundation on the arts & humanities",
    "natl geospatial intelligence agency",
    "natl guard",
    "natl highway traffic safety administration (nhtsa)",
    "natl indian gaming commission",
    "natl institute of justice",
    "natl institute of mental health (nimh)",
    "natl institute of standards & technology (nist)",
    "natl institute on alcohol abuse & alcoholism (niaa)",
    "natl institutes of health (nih)",
    "natl labor relations board (nlrb)",
    "natl mediation board",
    "natl oceanic & atmospheric administration (noaa)",
    "natl park service (nps)",
    "natl science foundation (nsf)",
    "natl security agency (nsa)",
    "natl security council (nsc)",
    "natl technical information service",
    "natl telecommunications & information administration (ntia)",
    "natl transportation safety board (ntsb)",
    "navy, dept of",
    "none",
    "nuclear regulatory commission (nrc)",
    "occupational safety & health administration (osha)",
    "occupational safety & health review commission",
    "office for victims of crime",
    "office of administration",
    "office of faith-based & community initiatives",
    "office of government ethics (oge)",
    "office of justice program",
    "office of juvenile justice & delinquency prevention",
    "office of management & budget (omb)",
    "office of natl aids policy",
    "office of natl drug control policy (ndcp)",
    "office of personnel management (opm)",
    "office of policy development",
    "office of science & technology policy (ostp)",
    "office of special counsel",
    "office of surface mining reclamation & enforcement",
    "office of technology policy (otp)",
    "office of the comptroller of the currency (occ)",
    "office of the secretary of defense",
    "office of the vice president of the united states",
    "office of thrift supervision (ots)",
    "office on violence against women",
    "on-site inspection agency",
    "overseas private investment corp (opic)",
    "panama canal commission",
    "patent & trademark office (pto)",
    "peace corps",
    "pension & welfare benefits administration",
    "pension benefit guaranty corporation (pbgc)",
    "pentagon force protection agency",
    "pipeline & hazardous materials safety administration",
    "please do not use this one",
    "postal rate commission",
    "postal regulatory commission",
    "president of the u.s.",
    "program support center",
    "railroad retirement board",
    "research & special programs administration",
    "risk management agency",
    "securities & exchange commission (sec)",
    "selective service system",
    "senate",
    "small business administration (sba)",
    "smithsonian institution",
    "social security administration (ssa)",
    "st. lawrence seaway development corporation",
    "state justice institute",
    "state, dept of (dos)",
    "substance abuse & mental health services administration (samhsa)",
    "surface transportation board (stb)",
    "technology administration",
    "tennessee valley authority (tva)",
    "transportation security administration (tsa)",
    "transportation, dept of (dot)",
    "treasury, dept of",
    "u.s. agency for international development (usaid)",
    "u.s. architectural & transportation barriers compliance board (access)",
    "u.s. arms control & disarmament agency",
    "u.s. botanic garden",
    "u.s. chemical safety & hazard investigation board",
    "u.s. coast guard (uscg)",
    "u.s. commission of fine arts",
    "u.s. commission on civil rights",
    "u.s. commission on international religious freedom",
    "u.s. copyright office",
    "u.s. customs & border protection",
    "u.s. fish & wildlife service (usfws)",
    "u.s. forest service",
    "u.s. geological survey (usgs)",
    "u.s. immigration & customs enforcement (ice)",
    "u.s. information agency (usia)",
    "u.s. institute of peace",
    "u.s. international development cooperation agency",
    "u.s. international trade commission (itc)",
    "u.s. marines",
    "u.s. marshals service",
    "u.s. mint",
    "u.s. natl central bureau - intl criminal police organization",
    "u.s. postal service (usps)",
    "u.s. secret service",
    "u.s. trade & development agency (ustda)",
    "u.s. trade representative (ustr)",
    "undetermined",
    "usa freedom corps",
    "veterans affairs, dept of (va)",
    "veterans employment & training service",
    "vice president of the u.s.",
    "voice of america",
    "white house office",
]

In [ ]:
# lobbying_activity_usd
years = list(range(2004, 2025))

tmp_df = (
    df[["filing_year", "lobbying_activity_usd", "very_lumped_sector"]]
    .loc[(df.very_likely_ccs == 1)]  # & (cleaned_df.epa == 1)]
    .groupby(["very_lumped_sector", "filing_year"])
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
    .reset_index()
).copy(deep=True)
fig, ax = plt.subplots(1, 1, figsize=(17, 5))
ax = sns.histplot(
    data=tmp_df,
    x="filing_year",
    bins=[0.5 + x for x in years],
    hue="very_lumped_sector",
    shrink=0.9,
    weights="lobbying_activity_usd",
    multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
# ax.legend(ncols=3, loc="upper center")

In [ ]:
top_10 = [
    "OCCIDENTAL PETROLEUM",
    "SOUTHERN COMPANY",
    "EXXON",
    "AMERICAN CHEMISTRY COUNCIL",
    "EDISON ELECTRIC INSTITUTE",
    "SHELL",
    "CHEVRON",
    "DUKE",
    "BP AMERICA",
    "AMERICAN ELECTRIC POWER",
]
top_3 = ["EXXON", "OCCIDENTAL PETROLEUM", "SOUTHERN COMPANY"]
top_5 = [
    "OCCIDENTAL PETROLEUM",
    "SOUTHERN COMPANY",
    "EXXON",
    "AMERICAN CHEMISTRY COUNCIL",
    "EDISON ELECTRIC INSTITUTE",
]
top_df = (
    df[["lobbying_activity_usd", "n_entities_lobbied", "client_rename", "filing_year"]]
    .loc[(df.very_likely_ccs == 1) & (df.client_rename.isin(top_5))]
    .groupby(["filing_year", "client_rename"])
    .sum()
    .sort_values(by="lobbying_activity_usd", ascending=False)
)

years = list(range(2004, 2025))

fig, ax = plt.subplots(1, 1, figsize=(17, 5))
ax = sns.histplot(
    data=top_df,
    x="filing_year",
    bins=[0.5 + x for x in years],
    hue="client_rename",
    shrink=0.9,
    weights="lobbying_activity_usd",
    multiple="stack",
)
ax.set_xticks(years)
ax.set_xticklabels([str(x) for x in years], fontsize=12)
ax.set_axisbelow(True)
ax.yaxis.grid()
ax.set_ylabel("100 million USD", fontsize=14)
ax.set_xlabel("")
# ax.legend(ncols=3, loc="upper center")

In [ ]:
# top 50 pct companies chart
df[
    [
        "client_rename",
        "sector",
        "lobbying_activity_usd",
        "n_entities_lobbied",
        "pct_of_lobbying_dollars",
        "pct_of_entity_contacts",
    ]
].groupby(["client_rename"]).agg(
    {
        "lobbying_activity_usd": sum,
        "n_entities_lobbied": sum,
        "pct_of_lobbying_dollars": sum,
        "pct_of_entity_contacts": sum,
        "sector": "first",
    }
).sort_values(
    by="lobbying_activity_usd", ascending=False
).head(
    16
)  # .to_clipboard()

In [217]:
# top sectors chart
df[
    [
        "lumped_sector",
        "lobbying_activity_usd",
        "n_entities_lobbied",
        "pct_of_lobbying_dollars",
        "pct_of_entity_contacts",
    ]
].groupby(["lumped_sector"]).agg(
    {
        "lobbying_activity_usd": sum,
        "n_entities_lobbied": sum,
        "pct_of_lobbying_dollars": sum,
        "pct_of_entity_contacts": sum,
    }
).sort_values(
    by="lobbying_activity_usd", ascending=False
).to_clipboard()

In [ ]:
df[
    [
        "lumped_sector",
        "lobbying_activity_usd",
        "n_entities_lobbied",
        "pct_of_lobbying_dollars",
        "pct_of_entity_contacts",
    ]
].loc[(df.very_likely_ccs == 1) & (df.filing_year > 2018)].sum()

In [ ]:
27844 / (260 * 5.5)